# PlotBot



### Imports

In [1]:
# Core
from datetime import date, timedelta, datetime, timezone
from operator import itemgetter
import json
import re

# Logging
import sys
import logging
logging.basicConfig(
    format='%(asctime)s | %(levelname)s : %(message)s',
    stream=sys.stdout) # Needed in Jupyter. Comment out if only need to see in log file.

# Scheduler
import schedule

# Numpy, Pandas and PyPlot
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()
sns.set_style(
    'darkgrid', 
    {
        'axes.facecolor': '0.9',
        'axes.titlesize': 'x-large',
        'figure.titlesize': 'x-large',
    }
)

# API
import tweepy
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Own
from twitter_config import *

### Constants and Globals

In [2]:
SCAN_MINUTES = 1
PAGE_SIZE = 20
PAGES = 2
TODAY = date.today().strftime('%m/%d/%y')

_AUTH = tweepy.OAuthHandler(TWITTER_CONSUMER_KEY, TWITTER_CONSUMER_SECRET)
_AUTH.set_access_token(TWITTER_ACCESS_TOKEN, TWITTER_TOKEN_SECRET)

TWITTER_API = tweepy.API(_AUTH, parser=tweepy.parsers.JSONParser())
SENTIMENT_ANALYZER = SentimentIntensityAnalyzer()

TWITTER_USER='feng443'

DEBUG = True
LOG_FILE = 'PlotBot.log'

### Define PlotBot Class

In [3]:
class PlotBot(object):
    
    _analyzed = []
    _to_analyze = []
    _df = pd.DataFrame(index=pd.Index(
        list(range(-PAGE_SIZE * PAGES)),
        name='Tweets Ago'))
    
    @property
    def data(self):
        return self._df

    @property
    def logger(self):
        logger = logging.getLogger('PlotBot')
        if self._debug:
            logger.setLevel(logging.DEBUG)
        if self._log_file:
            logger.addHandler(logging.FileHandler(self._log_file))
        return logger
    
    def __init__(self, user=TWITTER_USER, debug=False, log_file=None,
                last_only=True):
        self._user = user
        self._re = re.compile('@PlotBot Analyze: @(\w{1,15})')
        self._debug = debug
        self._log_file = log_file
        self._last_only = last_only
        
    def _scan_tweets(self, user=TWITTER_USER, debug=True):
        self.logger.info('Scan tweets ...')
        self._tweets = []
        for tweet in TWITTER_API.user_timeline(user, count=PAGE_SIZE):
            targets = self._re.findall(tweet['text'])
            if targets:
                target = targets[0]
                if target not in self._analyzed:
                    if self._last_only and self._is_old_tweet(tweet):
                        self.logger.debug('is_old_tweet: {}'.format(self._is_old_tweet(tweet)))
                        self.logger.info('Skip older tweets.')
                        break
                    self._to_analyze.append(target)    
                    created_at = tweet['created_at']
                else:
                    self.logger.info(f'{target} already analyzed. Skip.')
    
    def _is_old_tweet(self, tweet):
        tweet_datetime = datetime.strptime(
            tweet['created_at'], "%a %b %d %H:%M:%S %z %Y")
        self.logger.debug(f'tweet time: {tweet_datetime}')
        now = datetime.now(timezone.utc) 
        self.logger.debug(f'now: {now}')
        return (now - tweet_datetime) > timedelta(minutes=SCAN_MINUTES * 2) # Times 2 to allow some processing time buffer
        
    def _analyze_all(self):
        if not self._to_analyze:
            self.logger.info('Nothing new.')
            return

        self.logger.info('Analyze all ...')
        while self._to_analyze:
            target = self._to_analyze.pop()
            self._analyzed.append(target)
            self._analyze(target)
            self._plot(target)
            self._tweet_out(target)
    
    def _analyze(self, target):
        self.logger.info(f'Analyze {target}')
        tweets_ago = 0
        for page in range(PAGES):
            for tweet in TWITTER_API.user_timeline(target, count=PAGE_SIZE, page=page):
                self._df.at[tweets_ago, 
                        f'@{target}'] = SENTIMENT_ANALYZER.polarity_scores(tweet['text'])['compound']
                tweets_ago -= 1
    
    def _plot(self, target):
        fig, ax = plt.subplots(figsize=(8, 6))
        self.data.plot.line(
            y=f'@{target}',
            marker='o',
            ax=ax,
            alpha=0.8,
        )
        plt.legend(
            title='Tweets',
            bbox_to_anchor=(1.25, 1)
        )
        plt.title(f'Sentiments Analysis of Tweet ({TODAY})',
            fontsize=16)
        plt.ylabel('Tweet Polarity')
        plt.gca().invert_xaxis()
        
        plt.savefig('plot_bot.png', bbox_inches='tight')
        plt.show()
        
    def _tweet_out(self, target):
        self.logger.info(f'Tweet out  ...{target}')
        TWITTER_API.update_with_media('plot_bot.png',
                                      f'Sentiment Analsyis of Tweets ({TODAY})')
        
    def listen(self):
        self._scan_tweets()
        self._analyze_all()

### Start the Daemon

In [4]:
schedule.clear()
bot = PlotBot(debug=DEBUG, log_file=LOG_FILE)
schedule.every(SCAN_MINUTES).minutes.do(bot.listen)
    
while True:
    schedule.run_pending()

2018-03-18 20:23:14,004 | INFO : Scan tweets ...
2018-03-18 20:23:14,325 | DEBUG : tweet time: 2018-03-18 23:51:20+00:00
2018-03-18 20:23:14,326 | DEBUG : now: 2018-03-19 00:23:14.326177+00:00
2018-03-18 20:23:14,328 | DEBUG : tweet time: 2018-03-18 23:51:20+00:00
2018-03-18 20:23:14,330 | DEBUG : now: 2018-03-19 00:23:14.330173+00:00
2018-03-18 20:23:14,332 | DEBUG : is_old_tweet: True
2018-03-18 20:23:14,334 | INFO : Skip older tweets.
2018-03-18 20:23:14,336 | INFO : Nothing new.
2018-03-18 20:23:24,338 | INFO : Scan tweets ...
2018-03-18 20:23:24,714 | DEBUG : tweet time: 2018-03-18 23:51:20+00:00
2018-03-18 20:23:24,717 | DEBUG : now: 2018-03-19 00:23:24.716653+00:00
2018-03-18 20:23:24,719 | DEBUG : tweet time: 2018-03-18 23:51:20+00:00
2018-03-18 20:23:24,722 | DEBUG : now: 2018-03-19 00:23:24.721156+00:00
2018-03-18 20:23:24,723 | DEBUG : is_old_tweet: True
2018-03-18 20:23:24,726 | INFO : Skip older tweets.
2018-03-18 20:23:24,729 | INFO : Nothing new.
2018-03-18 20:23:34,734 

2018-03-18 20:26:01,627 | INFO : Skip older tweets.
2018-03-18 20:26:01,631 | INFO : Nothing new.
2018-03-18 20:26:11,636 | INFO : Scan tweets ...
2018-03-18 20:26:11,983 | DEBUG : tweet time: 2018-03-18 23:41:26+00:00
2018-03-18 20:26:11,989 | DEBUG : now: 2018-03-19 00:26:11.988580+00:00
2018-03-18 20:26:11,995 | DEBUG : tweet time: 2018-03-18 23:41:26+00:00
2018-03-18 20:26:12,002 | DEBUG : now: 2018-03-19 00:26:12.001583+00:00
2018-03-18 20:26:12,008 | DEBUG : is_old_tweet: True
2018-03-18 20:26:12,015 | INFO : Skip older tweets.
2018-03-18 20:26:12,022 | INFO : Nothing new.
2018-03-18 20:26:22,028 | INFO : Scan tweets ...
2018-03-18 20:26:22,386 | DEBUG : tweet time: 2018-03-18 23:41:26+00:00
2018-03-18 20:26:22,392 | DEBUG : now: 2018-03-19 00:26:22.392495+00:00
2018-03-18 20:26:22,400 | DEBUG : tweet time: 2018-03-18 23:41:26+00:00
2018-03-18 20:26:22,405 | DEBUG : now: 2018-03-19 00:26:22.405491+00:00
2018-03-18 20:26:22,412 | DEBUG : is_old_tweet: True
2018-03-18 20:26:22,419 

2018-03-18 20:28:59,283 | DEBUG : is_old_tweet: True
2018-03-18 20:28:59,293 | INFO : Skip older tweets.
2018-03-18 20:28:59,302 | INFO : Nothing new.
2018-03-18 20:29:09,312 | INFO : Scan tweets ...
2018-03-18 20:29:09,655 | DEBUG : tweet time: 2018-03-18 23:41:26+00:00
2018-03-18 20:29:09,664 | DEBUG : now: 2018-03-19 00:29:09.664010+00:00
2018-03-18 20:29:09,675 | DEBUG : tweet time: 2018-03-18 23:41:26+00:00
2018-03-18 20:29:09,683 | DEBUG : now: 2018-03-19 00:29:09.683003+00:00
2018-03-18 20:29:09,691 | DEBUG : is_old_tweet: True
2018-03-18 20:29:09,699 | INFO : Skip older tweets.
2018-03-18 20:29:09,707 | INFO : Nothing new.
2018-03-18 20:29:19,715 | INFO : Scan tweets ...
2018-03-18 20:29:20,089 | DEBUG : tweet time: 2018-03-18 23:41:26+00:00
2018-03-18 20:29:20,096 | DEBUG : now: 2018-03-19 00:29:20.096343+00:00
2018-03-18 20:29:20,105 | DEBUG : tweet time: 2018-03-18 23:41:26+00:00
2018-03-18 20:29:20,115 | DEBUG : now: 2018-03-19 00:29:20.115344+00:00
2018-03-18 20:29:20,124 

2018-03-18 20:31:57,839 | DEBUG : now: 2018-03-19 00:31:57.838970+00:00
2018-03-18 20:31:57,849 | DEBUG : is_old_tweet: True
2018-03-18 20:31:57,859 | INFO : Skip older tweets.
2018-03-18 20:31:57,871 | INFO : Nothing new.
2018-03-18 20:32:07,881 | INFO : Scan tweets ...
2018-03-18 20:32:08,282 | DEBUG : tweet time: 2018-03-18 23:41:26+00:00
2018-03-18 20:32:08,290 | DEBUG : now: 2018-03-19 00:32:08.289665+00:00
2018-03-18 20:32:08,302 | DEBUG : tweet time: 2018-03-18 23:41:26+00:00
2018-03-18 20:32:08,311 | DEBUG : now: 2018-03-19 00:32:08.311133+00:00
2018-03-18 20:32:08,322 | DEBUG : is_old_tweet: True
2018-03-18 20:32:08,334 | INFO : Skip older tweets.
2018-03-18 20:32:08,355 | INFO : Nothing new.
2018-03-18 20:32:18,369 | INFO : Scan tweets ...
2018-03-18 20:32:18,829 | DEBUG : tweet time: 2018-03-18 23:41:26+00:00
2018-03-18 20:32:18,837 | DEBUG : now: 2018-03-19 00:32:18.836703+00:00
2018-03-18 20:32:18,848 | DEBUG : tweet time: 2018-03-18 23:41:26+00:00
2018-03-18 20:32:18,858 

2018-03-18 20:34:56,862 | DEBUG : tweet time: 2018-03-18 23:41:26+00:00
2018-03-18 20:34:56,878 | DEBUG : now: 2018-03-19 00:34:56.877893+00:00
2018-03-18 20:34:56,895 | DEBUG : is_old_tweet: True
2018-03-18 20:34:56,912 | INFO : Skip older tweets.
2018-03-18 20:34:56,927 | INFO : Nothing new.
2018-03-18 20:35:06,943 | INFO : Scan tweets ...
2018-03-18 20:35:07,360 | DEBUG : tweet time: 2018-03-18 23:41:26+00:00
2018-03-18 20:35:07,371 | DEBUG : now: 2018-03-19 00:35:07.370739+00:00
2018-03-18 20:35:07,385 | DEBUG : tweet time: 2018-03-18 23:41:26+00:00
2018-03-18 20:35:07,400 | DEBUG : now: 2018-03-19 00:35:07.399739+00:00
2018-03-18 20:35:07,413 | DEBUG : is_old_tweet: True
2018-03-18 20:35:07,427 | INFO : Skip older tweets.
2018-03-18 20:35:07,441 | INFO : Nothing new.
2018-03-18 20:35:17,456 | INFO : Scan tweets ...
2018-03-18 20:35:17,869 | DEBUG : tweet time: 2018-03-18 23:41:26+00:00
2018-03-18 20:35:17,881 | DEBUG : now: 2018-03-19 00:35:17.880415+00:00
2018-03-18 20:35:17,894 

...

2018-03-18 23:20:42,515 | DEBUG : now: 2018-03-19 03:20:42.514440+00:00
2018-03-18 23:20:42,723 | DEBUG : is_old_tweet: True
2018-03-18 23:20:42,921 | INFO : Skip older tweets.
2018-03-18 23:20:43,155 | INFO : Nothing new.
2018-03-18 23:20:53,349 | INFO : Scan tweets ...
2018-03-18 23:20:53,839 | DEBUG : tweet time: 2018-03-18 23:41:26+00:00
2018-03-18 23:20:54,025 | DEBUG : now: 2018-03-19 03:20:54.023815+00:00
2018-03-18 23:20:54,265 | DEBUG : tweet time: 2018-03-18 23:41:26+00:00
2018-03-18 23:20:54,495 | DEBUG : now: 2018-03-19 03:20:54.494315+00:00
2018-03-18 23:20:54,703 | DEBUG : is_old_tweet: True
2018-03-18 23:20:54,894 | INFO : Skip older tweets.
2018-03-18 23:20:55,138 | INFO : Nothing new.
2018-03-18 23:21:05,373 | INFO : Scan tweets ...
2018-03-18 23:21:06,205 | DEBUG : tweet time: 2018-03-18 23:41:26+00:00
2018-03-18 23:21:06,415 | DEBUG : now: 2018-03-19 03:21:06.414221+00:00
2018-03-18 23:21:06,695 | DEBUG : tweet time: 2018-03-18 23:41:26+00:00
2018-03-18 23:21:06,889 

2018-03-18 23:24:10,993 | DEBUG : tweet time: 2018-03-18 23:41:26+00:00
2018-03-18 23:24:11,207 | DEBUG : now: 2018-03-19 03:24:11.205505+00:00
2018-03-18 23:24:11,397 | DEBUG : is_old_tweet: True
2018-03-18 23:24:11,590 | INFO : Skip older tweets.
2018-03-18 23:24:11,815 | INFO : Nothing new.
2018-03-18 23:24:22,060 | INFO : Scan tweets ...
2018-03-18 23:24:22,634 | DEBUG : tweet time: 2018-03-18 23:41:26+00:00
2018-03-18 23:24:22,865 | DEBUG : now: 2018-03-19 03:24:22.864894+00:00
2018-03-18 23:24:23,116 | DEBUG : tweet time: 2018-03-18 23:41:26+00:00
2018-03-18 23:24:23,312 | DEBUG : now: 2018-03-19 03:24:23.311399+00:00
2018-03-18 23:24:23,502 | DEBUG : is_old_tweet: True
2018-03-18 23:24:23,680 | INFO : Skip older tweets.
2018-03-18 23:24:23,871 | INFO : Nothing new.
2018-03-18 23:24:34,072 | INFO : Scan tweets ...
2018-03-18 23:24:34,721 | DEBUG : tweet time: 2018-03-18 23:41:26+00:00
2018-03-18 23:24:34,927 | DEBUG : now: 2018-03-19 03:24:34.926047+00:00
2018-03-18 23:24:35,138 

2018-03-18 23:27:38,266 | DEBUG : now: 2018-03-19 03:27:38.264961+00:00
2018-03-18 23:27:38,521 | DEBUG : tweet time: 2018-03-18 23:41:26+00:00
2018-03-18 23:27:38,722 | DEBUG : now: 2018-03-19 03:27:38.721961+00:00
2018-03-18 23:27:38,915 | DEBUG : is_old_tweet: True
2018-03-18 23:27:39,119 | INFO : Skip older tweets.
2018-03-18 23:27:39,318 | INFO : Nothing new.
2018-03-18 23:27:49,522 | INFO : Scan tweets ...
2018-03-18 23:27:50,054 | DEBUG : tweet time: 2018-03-18 23:41:26+00:00
2018-03-18 23:27:50,332 | DEBUG : now: 2018-03-19 03:27:50.330597+00:00
2018-03-18 23:27:50,564 | DEBUG : tweet time: 2018-03-18 23:41:26+00:00
2018-03-18 23:27:50,763 | DEBUG : now: 2018-03-19 03:27:50.762100+00:00
2018-03-18 23:27:50,991 | DEBUG : is_old_tweet: True
2018-03-18 23:27:51,308 | INFO : Skip older tweets.
2018-03-18 23:27:51,572 | INFO : Nothing new.
2018-03-18 23:28:01,830 | INFO : Scan tweets ...
2018-03-18 23:28:02,413 | DEBUG : tweet time: 2018-03-18 23:41:26+00:00
2018-03-18 23:28:02,651 

2018-03-18 23:31:07,597 | DEBUG : tweet time: 2018-03-18 23:41:26+00:00
2018-03-18 23:31:07,877 | DEBUG : now: 2018-03-19 03:31:07.875845+00:00
2018-03-18 23:31:08,508 | DEBUG : tweet time: 2018-03-18 23:41:26+00:00
2018-03-18 23:31:08,699 | DEBUG : now: 2018-03-19 03:31:08.698345+00:00
2018-03-18 23:31:08,877 | DEBUG : is_old_tweet: True
2018-03-18 23:31:09,081 | INFO : Skip older tweets.
2018-03-18 23:31:09,276 | INFO : Nothing new.
2018-03-18 23:31:19,458 | INFO : Scan tweets ...
2018-03-18 23:31:20,076 | DEBUG : tweet time: 2018-03-18 23:41:26+00:00
2018-03-18 23:31:20,284 | DEBUG : now: 2018-03-19 03:31:20.283677+00:00
2018-03-18 23:31:20,554 | DEBUG : tweet time: 2018-03-18 23:41:26+00:00
2018-03-18 23:31:20,779 | DEBUG : now: 2018-03-19 03:31:20.777678+00:00
2018-03-18 23:31:21,037 | DEBUG : is_old_tweet: True
2018-03-18 23:31:21,508 | INFO : Skip older tweets.
2018-03-18 23:31:22,113 | INFO : Nothing new.
2018-03-18 23:31:32,439 | INFO : Scan tweets ...
2018-03-18 23:31:33,087 

OSError: [Errno 24] Too many open files: 'C:\\Users\\feng443\\OneDrive\\Data Science Bootcamp\\git\\RUDSWeek7SocialAnalytics\\PlotBot.log'